<a href="https://colab.research.google.com/github/ccarpenterg/LearningTensorFlow2.0/blob/master/Gettingstarted_TensorFlow2_0_Intro_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction to Convolutional Neural Networks with TensorFlow

In [0]:
!pip install tensorflow==2.0.0-alpha0

In [0]:
#import print function from future
from __future__ import absolute_import, division, print_function, unicode_literals

#import TensorFlow and check version
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

print(tf.__version__)

2.0.0-alpha0


**MNIST Fashion Dataset**

Downloading and feeding the dataset to the neural network is too simple. The dataset is conveniently included in the ketas module:

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


### Neural Network Bookkeeping: Parameters

Now let's find out how the TensorFlow calculated the parameters for each convolutional layer. 

***1st convolutional layer***

In the first convolutional layer we have 32 kernels or filters of 3x3. 
Each filter has 9 (3x3) parameters and that gives us a total of 288 parameters: 32 x 9.
But at the same time, each one of these filters has a bias: 32 x 1

32 x 9 weights + 32 x 1 biases = 320 parameters

***2nd convolutional layer***

After applying the first pooling we end up with 32 feature maps, and that's the input for our second convolutional layer. In the second layer we use 64 filters of 3x3, but now we apply these 64 filter to each of the 32 feature maps.

Each filter has 9 (3x3) parameters and that gives us a total of 576 parameters: 64 x 9. And each one of the filters has a bias: 64  x 1

32 x (64 x 9 weights) + 64 x 1 biases = 18496 parameters.

***3rd convolutional layer***

After applying the second polling we end up with 64 feature maps. In the third layer we also use 64 filters of 3x3, and we apply these filters to the 64 feature maps.

Each filter has 9 (3x3) parameters and that gives us a total of 576 parameters: 64 x 9. And each one of the filters has a bias: 64  x 1

64 x (64 x 9 weights) + 64 x 1 biases = 36928 parameters.

Classifier

We now add a dense layer and final layer with 10 neurons, one for each of the digit classes (one, two, three, etc). Because of the way in which dense layers work we need to convert the 3D tensor into a vector, and for that task we use a Flatten layer.

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

#Let's check our neural network architecture again
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

Now let's do the math for the rest of the neural network. First we have to stretch out the output tensor of our 3rd convolution layer:

tensor dimensions = (3, 3, 64) --> 3 x 3 x 64 = 576 elements vector

That's the input layer dimension of the second part of our neural network. So now we have a vector of length 576, a hidden dense layer composed of 64 neurons and an output layer of 10 neurons.

Each of the dense layer's neurons are connected to all the neurons in the input layer. And each of the 64 neurones has a bias:

64 x 576 weights + 64 x 1 bias = 36928 parameters

And finally, for our output layer we have:

10 * 64 weights + 10 biases = 640 parameters
